In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import serial
import joblib
from pickle import load
from plotly.offline import iplot 
import plotly.graph_objects as go
import cufflinks as cf
cf.go_offline()

In [2]:
scaler = load(open('scaler_ver2.pkl', 'rb'))
model = joblib.load('./xgb_model_ver2.pkl')

In [3]:
def scalerData(df):
    return scaler.transform(df)

In [4]:
def round3(x):
    return round(x,3)

In [5]:
def sortData(x,y,z):
    slist = x+y+z
    aa = np.array(slist).reshape(1,-1)
    return scalerData(aa)

In [19]:
ser = serial.Serial() # 시리얼을 연결한다.
ser.port = 'COM3'
ser.baudrate = 115200 

# baudrate를 모른다면 연결된 serial을 불러와서 확인할 수 있다.
# print(ser)

ser.timeout = 1 #시리얼에 데이터를 불러올 때 지정하는 딜레이

# 시리얼을 열어준다.
ser.open()
a=[]
allG = []
allX = []
allY= []
allG1 = []
allX2 = []
allY3= []
aa=[]
Ecount = 0
n_count = 0
# 반복해서 데이터를 출력하기 위해 while 을 만들어주었다.
while ser.readable():
    try:
        data = ser.readline() 
        if len(data) > 20:
             # serial에 출력되는 데이터를 읽어준다.
            data = str(data,'utf-8')
            data1 = list(map(round3,map(float,data.split(','))))
            Ecount = 0
    #         print(float(data1[0].strip()))
            allG1.append(data1[0])
            allX2.append(data1[1])
            allY3.append(data1[2])
            allG.append(data1[0])
            allX.append(data1[1])
            allY.append(data1[2])
            a.append(0)
            if len(allG) > 30:

                if list(model.predict(sortData(allG[-30:], allX[-30:], allY[-30:])))[0] == 1:
                    n_count +=1
                    if n_count == 1:
                        del a[0]
                        a.append(1)
                        print("낙상")
#                         break
                else:
                    n_count = 0
        else:
            Ecount = Ecount+1
 
    except:
        Ecount = Ecount+1
#         print("Error")
    if (Ecount == 5):
        print("End")
        break
    #저장되는 데이터를 확인하고 data3의 크기가 10을 넘어가면 while을 끝낸다.
ser.close() 
angle = {"AllG": allG,"AllX": allX ,"AllY": allY , "line" :a}
# angle = {"AccX": AccX,"AccY": AccY ,"AccZ": AccZ, "GroX": groX,"GroY": groY,"GroZ": groZ}

df = pd.DataFrame(angle)
df1 = df.iloc[30:,0:3]
plt.rcParams["figure.figsize"] = (10,8)
fig = df1.iplot(asFigure=True)
for i in df[df.line==1].index:
    fig.add_trace(go.Scatter(x=[i,i], y=[min(df.AllX),max(df.AllG)],line_color='red',text = f'{i}'))
#     fig.add_scatter(x=[i]*100, y=np.linspace(min(df.AllX),max(df.AllG),100), name=f'{i}')
fig.show()


낙상
낙상
End
